In [1]:
# import pandas as pd
# data = pd.read_excel('附件一（训练集）.xlsx')

In [2]:
# import pandas as pd 
# data = pd.read_excel('附件一（训练集）.xlsx', sheet_name=None)


In [3]:
# merged_data = pd.concat(data.values(), ignore_index=True)
# # merged_data = data
# len(merged_data)

In [4]:
# merged_data.to_csv('merged_4.csv')

In [5]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
import torch.nn.functional as F


In [6]:
from scipy.stats import skew

def calculate_symmetry(signal):
    # Ensure the signal contains only numeric values, handling non-numeric values and NaN
    signal = pd.to_numeric(signal, errors='coerce')


    # Ensure there are enough data points to calculate symmetry
    if len(signal) == 0:
        return np.nan, np.nan

    # Calculate skewness
    signal_skewness = skew(signal)

    # Calculate center index
    center_index = len(signal) // 2
    left_part = signal[:center_index]
    right_part = signal[center_index:]

    # Compute the means of the left and right parts
    left_mean = np.mean(left_part)
    right_mean = np.mean(right_part)

    # Calculate symmetry measure (difference between left and right means)
    symmetry_measure = left_mean - right_mean

    return signal_skewness, symmetry_measure

In [7]:
def normalize(data):
    min_val = np.min(data)
    max_val = np.max(data)
    normalized_data = (data - min_val) / (max_val - min_val)
    return normalized_data

In [8]:
def extract_frequency_features(signal, fs):
    # 进行FFT
    signal = pd.to_numeric(signal)
    fft_result = np.fft.fft(signal)
    # print("fs:", fs)  # 或者使用 assert fs != 0
    if fs == 0:
        fs = 50_000
    freqs = np.fft.fftfreq(len(signal), d=1/fs)
    # 取前半部分频率和幅值
    half_length = len(signal) // 2
    magnitude = np.abs(fft_result[:half_length])
    frequency = freqs[:half_length]

    # 提取特征
    max_freq = frequency[np.argmax(magnitude)]  # 最大幅值对应的频率
    mean_mag = np.mean(magnitude)  # 幅值均值
    std_mag = np.std(magnitude)    # 幅值标准差
    energy = np.sum(magnitude ** 2) # 能量

    # 返回特征
    return max_freq, mean_mag, std_mag, energy

In [9]:
def calculate_slope(signal):
    slope = np.zeros_like(signal)  # 创建与信号相同长度的数组
    slope[:-1] = np.diff(signal)  # 前向差分
    return slope

In [10]:
def calculate_duty_cycle(signal, threshold=0):
    # 找到信号大于阈值的部分
    high_time = np.sum(signal > threshold)  # 高电平时间
    total_time = len(signal)  # 总时间 (信号的长度)

    # 计算占空比
    duty_cycle = (high_time / total_time) * 100  # 百分比形式

    return duty_cycle

In [11]:
def extract_features(row):
    flux_density = row.iloc[4:]  # 假设磁通密度列从第5列开始
    # print(type(flux_density.values))
    # print(flux_density.values)
    # print(len(flux_density.values))
    fs = row.iloc[1]
    skewness, symmetric_messure = calculate_symmetry(flux_density.values)
    max_freq, mean_mag, std_mag, energy = extract_frequency_features(flux_density.values, fs)
    slope = calculate_slope(flux_density.values)
    return pd.Series({
        # 'mean_flux': np.mean(flux_density),
        # 'std_flux': np.std(flux_density),
        'max_flux': np.max(flux_density),
        # 'min_flux': np.min(flux_density),
        # 'range_flux': np.max(flux_density) - np.min(flux_density),
        # 'skewness': skewness,
        # 'symmetric_messure': symmetric_messure,
        # 'max_freq': max_freq,
        # 'mean_mag': mean_mag,
        # 'std_mag': std_mag,
        # 'energy': energy,
        # 'slope_mean': np.mean(slope),
        # 'slope_std': np.std(slope),
        # 'slope_max': np.max(slope),
        # 'slope_min': np.min(slope),
        # 'slope_abs_mean': np.mean(np.abs(slope)),
        # 'duty_cicle': calculate_duty_cycle(flux_density.values),
    })

# features = merged_data.apply(extract_features, axis=1)

In [1]:
import pandas as pd
features_lst = []
for i in range(1, 5):
    filename = '材料' + str(i)
    data = pd.read_excel('附件一（训练集）.xlsx', sheet_name=filename)
    # data = data[data['wav'] == '正弦波']
    # features = data.apply(extract_features, axis=1)
    # features_normalized = (features - features.min()) / (features.max() - features.min()) * 2 -1
    # features['label'] = data['励磁波形']
    # features['温度'] = data['温度，oC']
    # features['频率Hz'] = data['频率，Hz']
    # features['损耗'] =data['磁芯损耗，w/m3']
    # features.to_csv('材料'+ str(i)+'特征.csv', index=False, encoding='utf-8-sig')
    features_lst.append(data)

#dd 

In [2]:
features_lst[1][1000:1002]

,温度，oC,频率，Hz,磁芯损耗，w/m3,励磁波形,0（磁通密度B，T）,1,2,3,4,5,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
1000,70,251170,1874.30,正弦波,0.00086,0.00078,0.00071,0.00063,0.00056,0.00048,...,0.00162,0.00154,0.00146,0.00139,0.00131,0.00124,0.00116,0.00109,0.00101,0.00093
1001,70,50020,3784.87,正弦波,0.01813,0.01842,0.01870,0.01899,0.01927,0.01955,...,0.01524,0.01553,0.01582,0.01612,0.01641,0.01670,0.01699,0.01727,0.01756,0.01785


In [3]:
result = pd.concat(features_lst, ignore_index=True)

In [4]:
len(result)

12400

In [5]:
all_train = pd.read_csv('all_train.csv')
all_train['core_loss'] = result['磁芯损耗，w/m3']
all_train[:2]

,temperature,frequency,core_loss,waveform,0,1,2,3,4,5,...,1015,1016,1017,1018,1019,1020,1021,1022,1023,material
0,25,50030,1997.95525,正弦波,0.000213,0.000389,0.000566,0.000743,0.000919,0.001096,...,-0.001374,-0.001198,-0.001022,-0.000846,-0.000669,-0.000492,-0.000316,-0.000140,0.000036,材料1
1,25,50020,2427.74983,正弦波,-0.000551,-0.000358,-0.000165,0.000028,0.000221,0.000413,...,-0.002284,-0.002091,-0.001899,-0.001707,-0.001515,-0.001322,-0.001129,-0.000937,-0.000744,材料1


In [7]:
len(all_train)
all_train.to_csv('all_train_full_loss.csv', encoding='utf-8-sig', index=False)

In [36]:
result.to_csv("result.csv")

In [28]:
data1 = pd.read_excel('附件一（训练集）.xlsx', sheet_name='材料1')
features = data1.apply(extract_features, axis=1)

In [5]:
from sklearn.model_selection import train_test_split
import pandas as pd
# 定义一个函数来分割数据
def split_data(group):
    return train_test_split(group, test_size=0.2, random_state=42)
for i in range(1, 5):
    filename = f'材料{str(i)}特征.csv'
    data = pd.read_csv(filename)
    # 使用 groupby 对温度分组，并使用 apply 执行 split_data
    split_results = data.groupby('温度', group_keys=False).apply(lambda x: split_data(x)).reset_index(drop=True)

    # 分别取出 80% 和 20% 的数据
    df_80 = pd.concat([result[0] for result in split_results])
    df_20 = pd.concat([result[1] for result in split_results])

    print(f"Original {len(data)}")
    # 输出结果
    print("80% DataFrame:")
    print(len(df_80))
    # print(df_80[:2])
    print("\n20% DataFrame:")
    print(len(df_20))
    df_80.to_csv(f'材料{str(i)}特征_train.csv', index=False, encoding='utf-8-sig')
    df_20.to_csv(f'材料{str(i)}特征_test.csv', index=False, encoding='utf-8-sig')
    # print(df_20[:2])


Original 1067
80% DataFrame:
852

20% DataFrame:
215
Original 1097
80% DataFrame:
876

20% DataFrame:
221
Original 1010
80% DataFrame:
807

20% DataFrame:
203
Original 880
80% DataFrame:
701

20% DataFrame:
179


/tmp/ipykernel_701403/646051103.py:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  split_results = data.groupby('温度', group_keys=False).apply(lambda x: split_data(x)).reset_index(drop=True)
/tmp/ipykernel_701403/646051103.py:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  split_results = data.groupby('温度', group_keys=False).apply(lambda x: split_data(x)).reset_index(drop=True)
/tmp/ipykernel_701403/6460